<a href="https://colab.research.google.com/github/Rito43/Rito43/blob/main/M23CSA021_DLOps_ClassAssignment_3_ipynb_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ray

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 12.1 MB/s eta 0:00:00


In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models
from ray import tune
from ray.tune.schedulers import ASHAScheduler

In [ ]:
pip install -U tensorboardx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.1 MB/s eta 0:00:00


In [ ]:


# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load STL10 dataset
trainset = datasets.STL10(root='./data', split='train', download=True, transform=transform)
testset = datasets.STL10(root='./data', split='test', download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

# Define ResNet18 model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the training function
def train_cnn(config):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=0.9)

    for epoch in range(5):  # Example: 5 epochs for simplicity
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(trainloader)
        train_accuracy = 100.0 * correct / total

        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.3f}, Train Accuracy: {train_accuracy:.2f}%")

# Define the hyperparameter search space
config = {
    "lr": tune.loguniform(1e-6, 1e-1),
    "batch_size": tune.choice([4, 8, 16, 32])
}

# Initialize TensorBoard
writer = SummaryWriter()

# Define the scheduler
scheduler = ASHAScheduler(
    metric="train_loss",
    mode="min",
    max_t=5,
    grace_period=1,
    reduction_factor=2)

# Perform hyperparameter tuning
analysis = tune.run(
    train_cnn,
    config=config,
    resources_per_trial={"cpu": 2, "gpu": 1},
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=tune.CLIReporter(),
    name="resnet18_stl10_tuning"
)

# Get best hyperparameters and train the model with them
best_config = analysis.get_best_config(metric="train_loss", mode="min")
best_lr = best_config["lr"]
best_batch_size = best_config["batch_size"]

# Train model with best hyperparameters
optimizer = optim.SGD(model.parameters(), lr=best_lr, momentum=0.9)
train_cnn(best_config)

# Save the model
torch.save(model.state_dict(), "resnet18_stl10_model.pth")

# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

top5_test_accuracy = 100 * correct / total
print(f"Top-5 Test Accuracy: {top5_test_accuracy:.2f}%")

# Close TensorBoard writer
writer.close()


Files already downloaded and verified
Files already downloaded and verified


2024-03-31 10:14:20,956	INFO worker.py:1752 -- Started a local Ray instance.
2024-03-31 10:14:22,648	INFO tune.py:263 -- Initializing Ray automatically. For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run(...)`.
2024-03-31 10:14:22,660	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 2. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-03-31 10:14:25,308	INFO tensorboardx.py:196 -- pip install "ray[tune]" to see TensorBoard files.
2024-03-31 10:14:25,310	WARNING callback.py:137 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`
2024-03-31 10:14:25,316	WARNING tune.py:912 -- AIR_VERBOSITY is set, ignoring p

+----------------------------------------------------------+
| Configuration for experiment     resnet18_stl10_tuning   |
+----------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator   |
| Scheduler                        AsyncHyperBandScheduler |
| Number of trials                 10                      |
+----------------------------------------------------------+

View detailed results here: /root/ray_results/resnet18_stl10_tuning
(autoscaler +8m35s) Tip: use `ray status` to view detailed cluster status. To disable these messages, set RAY_SCHEDULER_EVENTS=0.
(autoscaler +8m35s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.

Trial status: 10 PENDING
Current time: 2024-03-31 10:15:13. Total running time: 48s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------+
| Trial nam

2024-03-31 10:16:13,836	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 2.0 CPUs and 1.0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


Trial status: 10 PENDING
Current time: 2024-03-31 10:16:13. Total running time: 1min 48s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------+
| Trial name              status              lr     batch_size |
+---------------------------------------------------------------+
| train_cnn_73698_00000   PENDING    1.65504e-06             32 |
| train_cnn_73698_00001   PENDING    1.18922e-06              4 |
| train_cnn_73698_00002   PENDING    0.0343207                8 |
| train_cnn_73698_00003   PENDING    0.0283974                4 |
| train_cnn_73698_00004   PENDING    0.00433067              32 |
| train_cnn_73698_00005   PENDING    0.00082331              16 |
| train_cnn_73698_00006   PENDING    0.00174239              32 |
| train_cnn_73698_00007   PENDING    2.59099e-05              4 |
| train_cnn_73698_00008   PENDING    0.0515474               16 |
| train_cnn_73698_00009   PENDING    0.00311901               4 |
+---------

...

2024-03-31 10:33:14,676	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 2.0 CPUs and 1.0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


Trial status: 10 PENDING
Current time: 2024-03-31 10:33:16. Total running time: 18min 51s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------+
| Trial name              status              lr     batch_size |
+---------------------------------------------------------------+
| train_cnn_73698_00000   PENDING    1.65504e-06             32 |
| train_cnn_73698_00001   PENDING    1.18922e-06              4 |
| train_cnn_73698_00002   PENDING    0.0343207                8 |
| train_cnn_73698_00003   PENDING    0.0283974                4 |
| train_cnn_73698_00004   PENDING    0.00433067              32 |
| train_cnn_73698_00005   PENDING    0.00082331              16 |
| train_cnn_73698_00006   PENDING    0.00174239              32 |
| train_cnn_73698_00007   PENDING    2.59099e-05              4 |
| train_cnn_73698_00008   PENDING    0.0515474               16 |
| train_cnn_73698_00009   PENDING    0.00311901               4 |
+--------

2024-03-31 10:33:59,385	WARNING tune.py:229 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-03-31 10:33:59,395	INFO tune.py:1016 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/resnet18_stl10_tuning' in 0.0068s.
2024-03-31 10:33:59,428	WARNING tune.py:1063 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: tune.run(..., resume=True)
2024-03-31 10:33:59,458	WARNING experiment_analysis.py:190 -- Failed to fetch metrics for 10 trial(s):
- train_cnn_73698_00000: FileNotFoundError('Could not fetch metrics for train_cnn_73698_00000: both result.json and progress.csv were not found at /root/ray_results/resnet18_stl10_tuning/train_cnn_73698_00000_0_batch_size=32,lr=0.0000_2024-03-31_10-14-53')
- train_cnn_73698_00001: FileNotFoundError('Could not fetch metrics 

Trial status: 10 PENDING
Current time: 2024-03-31 10:33:59. Total running time: 19min 34s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------+
| Trial name              status              lr     batch_size |
+---------------------------------------------------------------+
| train_cnn_73698_00000   PENDING    1.65504e-06             32 |
| train_cnn_73698_00001   PENDING    1.18922e-06              4 |
| train_cnn_73698_00002   PENDING    0.0343207                8 |
| train_cnn_73698_00003   PENDING    0.0283974                4 |
| train_cnn_73698_00004   PENDING    0.00433067              32 |
| train_cnn_73698_00005   PENDING    0.00082331              16 |
| train_cnn_73698_00006   PENDING    0.00174239              32 |
| train_cnn_73698_00007   PENDING    2.59099e-05              4 |
| train_cnn_73698_00008   PENDING    0.0515474               16 |
| train_cnn_73698_00009   PENDING    0.00311901               4 |
+--------

TypeError: 'NoneType' object is not subscriptable

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms, models
from ray import tune
from ray.tune.schedulers import ASHAScheduler

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Load STL10 dataset
trainset = datasets.STL10(root='./data', split='train', download=True, transform=transform)
testset = datasets.STL10(root='./data', split='test', download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=4, shuffle=False)

# Define ResNet18 model
model = models.resnet18(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the training function
def train_cnn(config):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config["lr"])

    for epoch in range(2):  # Example: 5 epochs for simplicity
        running_loss = 0.0
        correct = 0
        total = 0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        train_loss = running_loss / len(trainloader)
        train_accuracy = 100.0 * correct / total

        print(f"Epoch {epoch+1}, Train Loss: {train_loss:.3f}, Train Accuracy: {train_accuracy:.2f}%")

# Define the hyperparameter search space
config = {
    "lr": tune.loguniform(1e-6, 1e-1),
    "batch_size": tune.choice([4, 8, 16, 32])
}

# Initialize TensorBoard
writer = SummaryWriter()

# Define the scheduler
scheduler = ASHAScheduler(
    metric="train_loss",
    mode="min",
    max_t=5,
    grace_period=1,
    reduction_factor=2)

# Perform hyperparameter tuning
analysis = tune.run(
    train_cnn,
    config=config,
    resources_per_trial={"cpu": 2, "gpu": 1},
    num_samples=10,
    scheduler=scheduler,
    progress_reporter=tune.CLIReporter(),
    name="resnet18_stl10_tuning"
)

# Get best hyperparameters and train the model with them
best_config = analysis.get_best_config(metric="train_loss", mode="min")
best_lr = best_config["lr"]
best_batch_size = best_config["batch_size"]

# Train model with best hyperparameters
optimizer = optim.Adam(model.parameters(), lr=best_lr)
train_cnn(best_config)

# Save the model
torch.save(model.state_dict(), "resnet18_stl10_model.pth")

# Test the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

top5_test_accuracy = 100 * correct / total
print(f"Top-5 Test Accuracy: {top5_test_accuracy:.2f}%")

# Close TensorBoard writer
writer.close()


Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
2024-03-31 10:35:15,805	INFO tune.py:622 -- [output] This will use the new output engine with verbosity 2. To disable the new output and use the legacy output engine, set the environment variable RAY_AIR_NEW_OUTPUT=0. For more information, please see https://github.com/ray-project/ray/issues/36949
2024-03-31 10:35:20,110	WARNING tune.py:912 -- AIR_VERBOSITY is set, ignoring 

+----------------------------------------------------------+
| Configuration for experiment     resnet18_stl10_tuning   |
+----------------------------------------------------------+
| Search algorithm                 BasicVariantGenerator   |
| Scheduler                        AsyncHyperBandScheduler |
| Number of trials                 10                      |
+----------------------------------------------------------+

View detailed results here: /root/ray_results/resnet18_stl10_tuning
To visualize your results with TensorBoard, run: `tensorboard --logdir /tmp/ray/session_2024-03-31_10-14-16_142736_240/artifacts/2024-03-31_10-35-19/resnet18_stl10_tuning/driver_artifacts`
(autoscaler +29m29s) Error: No available node types can fulfill resource request {'CPU': 2.0, 'GPU': 1.0}. Add suitable node types to this cluster to resolve this issue.

Trial status: 10 PENDING
Current time: 2024-03-31 10:36:07. Total running time: 47s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+----------------

2024-03-31 10:37:07,841	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 2.0 CPUs and 1.0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


Trial status: 10 PENDING
Current time: 2024-03-31 10:37:07. Total running time: 1min 47s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------+
| Trial name              status              lr     batch_size |
+---------------------------------------------------------------+
| train_cnn_5f241_00000   PENDING    1.48597e-06             32 |
| train_cnn_5f241_00001   PENDING    5.32527e-06             32 |
| train_cnn_5f241_00002   PENDING    1.56613e-05              4 |
| train_cnn_5f241_00003   PENDING    6.90998e-06             16 |
| train_cnn_5f241_00004   PENDING    0.0227893               32 |
| train_cnn_5f241_00005   PENDING    0.0596623                4 |
| train_cnn_5f241_00006   PENDING    5.63186e-05             16 |
| train_cnn_5f241_00007   PENDING    2.88826e-06             16 |
| train_cnn_5f241_00008   PENDING    0.0371082                4 |
| train_cnn_5f241_00009   PENDING    0.0208827                4 |
+---------

...

Trial status: 10 PENDING
Current time: 2024-03-31 11:32:16. Total running time: 56min 56s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------+
| Trial name              status              lr     batch_size |
+---------------------------------------------------------------+
| train_cnn_5f241_00000   PENDING    1.48597e-06             32 |
| train_cnn_5f241_00001   PENDING    5.32527e-06             32 |
| train_cnn_5f241_00002   PENDING    1.56613e-05              4 |
| train_cnn_5f241_00003   PENDING    6.90998e-06             16 |
| train_cnn_5f241_00004   PENDING    0.0227893               32 |
| train_cnn_5f241_00005   PENDING    0.0596623                4 |
| train_cnn_5f241_00006   PENDING    5.63186e-05             16 |
| train_cnn_5f241_00007   PENDING    2.88826e-06             16 |
| train_cnn_5f241_00008   PENDING    0.0371082                4 |
| train_cnn_5f241_00009   PENDING    0.0208827                4 |
+--------

2024-03-31 11:33:11,543	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 2.0 CPUs and 1.0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


Trial status: 10 PENDING
Current time: 2024-03-31 11:33:16. Total running time: 57min 56s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------+
| Trial name              status              lr     batch_size |
+---------------------------------------------------------------+
| train_cnn_5f241_00000   PENDING    1.48597e-06             32 |
| train_cnn_5f241_00001   PENDING    5.32527e-06             32 |
| train_cnn_5f241_00002   PENDING    1.56613e-05              4 |
| train_cnn_5f241_00003   PENDING    6.90998e-06             16 |
| train_cnn_5f241_00004   PENDING    0.0227893               32 |
| train_cnn_5f241_00005   PENDING    0.0596623                4 |
| train_cnn_5f241_00006   PENDING    5.63186e-05             16 |
| train_cnn_5f241_00007   PENDING    2.88826e-06             16 |
| train_cnn_5f241_00008   PENDING    0.0371082                4 |
| train_cnn_5f241_00009   PENDING    0.0208827                4 |
+--------

2024-03-31 11:34:11,581	WARNING insufficient_resources_manager.py:163 -- Ignore this message if the cluster is autoscaling. No trial is running and no new trial has been started within the last 60 seconds. This could be due to the cluster not having enough resources available. You asked for 2.0 CPUs and 1.0 GPUs per trial, but the cluster only has 2.0 CPUs and 0 GPUs available. Stop the tuning and adjust the required resources (e.g. via the `ScalingConfig` or `resources_per_trial`, or `num_workers` for rllib), or add more resources to your cluster.


Trial status: 10 PENDING
Current time: 2024-03-31 11:34:16. Total running time: 58min 56s
Logical resource usage: 0/2 CPUs, 0/0 GPUs
+---------------------------------------------------------------+
| Trial name              status              lr     batch_size |
+---------------------------------------------------------------+
| train_cnn_5f241_00000   PENDING    1.48597e-06             32 |
| train_cnn_5f241_00001   PENDING    5.32527e-06             32 |
| train_cnn_5f241_00002   PENDING    1.56613e-05              4 |
| train_cnn_5f241_00003   PENDING    6.90998e-06             16 |
| train_cnn_5f241_00004   PENDING    0.0227893               32 |
| train_cnn_5f241_00005   PENDING    0.0596623                4 |
| train_cnn_5f241_00006   PENDING    5.63186e-05             16 |
| train_cnn_5f241_00007   PENDING    2.88826e-06             16 |
| train_cnn_5f241_00008   PENDING    0.0371082                4 |
| train_cnn_5f241_00009   PENDING    0.0208827                4 |
+--------